In [4]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed'
    }
# TSS_pre = []
# with open(dic['ATAC_hema_2'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         # if ll[0] in ['1']:
#         TSS_pre.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))

# TSS_pre = []
# with open(dic['ATAC_Bcell'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] in ['1'] and ll[2] == '6':
#             TSS_pre.append(TSS(ll[0],int(ll[1])))

TSS_pre = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] == '4':
            TSS_pre.append(TSS(ll[0],int(int(ll[1])+1000)))
non_pre = []
with open(dic['non_gene_4'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        non_pre.append(TSS(ll[0], int(ll[1])+int(1000)))
TSS_pre = TSS_pre + non_pre[:len(TSS_pre)]

bamfile = ps.AlignmentFile(dic['bam4'],'rb')

TSS_pre = np.array(TSS_pre)

In [5]:
up = 1000
down = 1000
# feature_matrix = []
# for j, tss in enumerate(TSS_pre):
#     chrom = tss.chrom
#     start = tss.pos - up
#     end = tss.pos + down
#     up_end = np.zeros(up+down, dtype= int)
#     down_end = np.zeros(up+down, dtype= int)
#     long = np.zeros(up+down, dtype= int)
#     short = np.zeros(up+down, dtype= int)
#     cov = np.zeros(up+down, dtype= int)
#     for r in bamfile.fetch(chrom[-1], start-500, end + 500):
#         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
#             if r.reference_start + abs(r.isize) < start:
#                 continue
#             if r.reference_start >= end:
#                 continue
#             # ss = max(0, r.reference_start - start)
#             # ee = min(r.reference_start - start + abs(r.isize), end - start)
#             ss = r.reference_start - start
#             ee = r.reference_start - start + abs(r.isize)
#             if ss >= 0:
#                 up_end[ss] += 1
#             else:
#                 ss = 0
#             if ee < end - start:
#                 down_end[ee] += 1
#             else:
#                 ee = end - start
#             for i in range(ss, ee):
#                 cov[i] += 1
#             if 200 >= abs(r.isize) > 130:
#                 for i in range(ss, ee):
#                     long[i] += 1
#             if abs(r.isize) <= 130:
#                 for i in range(ss, ee):
#                     short[i] += 1
#     k = 0
#     win = 50
#     feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
#     while k < (up+down)/win:
#         ss = k * win
#         ee = k * win + win
#         ff = []
#         ff.append(int(round(np.mean(cov[ss:ee]))))
#         ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
#         ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
#         wps = 0
#         for r in bamfile.fetch(chrom[-1], ss+start, ee+start):
#             if r.reference_start <= ss+start and r.reference_start+abs(r.isize) >= ee + start:
#                 wps += 1
#             elif ee + start > r.reference_start > ss + start:
#                 wps -= 1
#             elif ss+start < r.reference_start + abs(r.isize) < ee + start:
#                 wps -= 1
#         ff.append(int(wps))
#         feature_win[k] = np.array(ff)
#         k =  k + 1
#     feature_matrix.append(feature_win)
# feature_matrix = np.array(feature_matrix)
win = 40
feature_matrix = []
for j, tss in enumerate(TSS_pre):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    # k = 0
    # win = 40
    # feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    # while k < (up+down)/win:
    #     ss = k * win
    #     ee = k * win + win
    #     ff = []
    #     ff.append(int(round(np.mean(cov[ss:ee]))))
    #     ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
    #     ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
    #     ff.append(int(round(np.mean(wps[ss:ee]))))
    #     feature_win[k] = np.array(ff)
    #     k = k + 1
    # feature_matrix.append(feature_win)
    ff = [cov,long-short,abs(up_end-down_end),wps]
    feature_matrix.append([[row[i]for row in ff] for i in range(len(ff[0]))])
    
feature_matrix = np.array(feature_matrix)
# lstm_x = []
# for mat in feature_matrix:
#     lstm_x.append(mat)
# lstm_x = np.array(lstm_x)
lstm_x = feature_matrix

raw_data = []
for j, tss in enumerate(TSS_pre):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    distribution_matrix = np.zeros((int((up+down)/50), 40), dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//50,relative_isize//5] += 1
    raw_data.append(distribution_matrix)
raw_data = np.array(raw_data)
cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)

In [6]:
from keras.models import load_model
from keras import Model
model_cnn = load_model('./cnn.h5')
dense1_layer_model = Model(inputs=model_cnn.input, outputs=model_cnn.layers[-2].output)
dense1_layer_model.trainable = False
model_lstm = load_model('./lstm.h5')
dense2_layer_model = Model(inputs=model_lstm.input, outputs=model_lstm.layers[-2].output)
dense2_layer_model.trainable = False
pred_lstm_y = dense2_layer_model.predict(lstm_x)
pred_cnn_y = dense1_layer_model.predict(cnn_x)
pred_x = np.append(pred_lstm_y,pred_cnn_y,axis=1)

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import random
import joblib

test_x = pred_x
test_y = np.array([1]*len(test_x))

svcClf = joblib.load('./svm.pkl')
rfClf = joblib.load('./rf.pkl')
treeClf = joblib.load('./tree.pkl')

# pred_svm = svcClf.predict(test_x)
# num = len(pred_svm)
# accuracy_svm = len([1 for i in range(num) if test_y[i] == pred_svm[i]]) / float(num)
# print("svm,acc:",accuracy_svm)

pred_rf = rfClf.predict(test_x)
num = len(pred_rf)
accuracy_rf = len([1 for i in range(num) if test_y[i] == pred_rf[i]]) / float(num)
print("rf,acc:",accuracy_rf)

# pred_tree = treeClf.predict(test_x)
# num = len(pred_tree)
# accuracy_tree = len([1 for i in range(num) if test_y[i] == pred_tree[i]]) / float(num)
# print("tree,acc:",accuracy_tree)

# pred = []
# for i in range(len(pred_rf)):
#     if pred_svm[i] + pred_rf[i] + pred_tree[i] >= 2:
#         pred.append(1)
#     else:
#         pred.append(0)
# num = len(pred)
# accuracy = len([1 for i in range(num) if test_y[i] == pred[i]]) / float(num)
# print("jicheng, acc:",accuracy)


2021-11-24 19:12:20.371617: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-24 19:12:20.371661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-24 19:12:21.295132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-24 19:12:21.295173: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-24 19:12:21.295194: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jiay-virtual-machine): /proc/driver/nvidia/version does not exist
2021-11-24 19:12:21.295374: I tensorflow/core/platform/c

rf,acc: 0.47844827586206895


In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = keras.models.Sequential()
model.add(layers.Dense(300,activation='relu',kernel_regularizer=None, bias_regularizer=None,input_shape=(1,32)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=None, bias_regularizer=None))
model.load_weights('./lstm_cnn_dense.h5')
result = model.predict(test_x,verbose=0)

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test_y, result, pos_label=1)
auc = metrics.auc(fpr, tpr)


/home/jiay/.local/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:941: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn("No negative samples in y_true, "


In [9]:
from sklearn import metrics
tn,fp,fn,tp = metrics.confusion_matrix(pred_svm, test_y,labels=[0,1]).ravel()
print('sensitivity:',tp/(tp+fn))
print('specificity:',tn/(tn+fp))
print(pred_svm)

sensitivity: 1.0
specificity: 0.0
[1 1 0 ... 1 0 1]


analyze the model

In [44]:
from keras.models import load_model
from keras import Model
model_cnn = load_model('./cnn.h5')
dense1_layer_model = Model(inputs=model_cnn.input, outputs=model_cnn.layers[-2].output)
dense1_layer_model.trainable = False
model_lstm = load_model('./lstm.h5')
dense2_layer_model = Model(inputs=model_lstm.input, outputs=model_lstm.layers[-2].output)
dense2_layer_model.trainable = False
pred_lstm_y = dense2_layer_model.predict(lstm_x)
pred_cnn_y = dense1_layer_model.predict(cnn_x)

pred_x = np.append(pred_lstm_y,pred_cnn_y,axis=1)

In [51]:
positive = pred_lstm_y[:int(len(pred_cnn_y)/2)]
negetive = pred_lstm_y[int(len(pred_cnn_y)/2):]
import pandas as pd
df1 = pd.DataFrame(positive)
df2 = pd.DataFrame(negetive)
df1.to_csv('./df11.csv')
df2.to_csv('./df22.csv')